In [1]:
import os

# Show current mask (None → all GPUs visible)
print("CUDA_VISIBLE_DEVICES was:", os.getenv("CUDA_VISIBLE_DEVICES"))

# ❏ Only use physical GPU 1  (host index)  ──────────────────────────
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print("CUDA_VISIBLE_DEVICES now:", os.environ["CUDA_VISIBLE_DEVICES"])


CUDA_VISIBLE_DEVICES was: None
CUDA_VISIBLE_DEVICES now: 1


In [2]:
!pip install "numpy<2.0"        # gets wheel 1.26.4 (glibc ≥ 2.17)
!pip install ultralytics
!pip install scikit-learn

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [3]:
import ultralytics
import glob
import shutil, yaml
import random
from pathlib import Path
from collections import Counter
import re, os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix

In [4]:
#set seed
random.seed(42)

root_dir = "/mnt/sdb/home/si25_osts0525/UAV_Inspection_System" #fix to your root directory
data_dir = root_dir + "/dataset"
source_data_yaml = os.path.join(data_dir, "source/data.yaml")
target_data_yaml = os.path.join(data_dir, "target/data.yaml")

#create results directories
source_result_dir = os.path.join(root_dir, "source_results")
target_result_dir = os.path.join(root_dir, "target_results")
src_eval_result_dir = os.path.join(root_dir, "test_results/Pretrained")
tar_eval_result_dir = os.path.join(root_dir, "test_results/Target")
os.makedirs(source_result_dir, exist_ok=True)
os.makedirs(target_result_dir, exist_ok=True)
os.makedirs(src_eval_result_dir, exist_ok=True)
os.makedirs(tar_eval_result_dir, exist_ok=True)

resume_model = ""
checkpoint = 0
global checkpoint

In [40]:
def training_pipeline(CVmodel, src_yaml, tar_yaml, 
                      src_eps = 150, tar_eps = 150, fine_tune_eps = 100, imgsiz = 640, 
                      resume = False, resume_directory = ""):
    global checkpoint
    model_name = CVmodel.split(".")[0]

    # #-------------------------------------Pretain the model on the SOURCE dataset -------------------------------------
    # model = YOLO()
    # if resume == True and CVmodel == resume_model and checkpoint == 1:
    #     print("Resume training model on SOURCE set...")
    #     model = YOLO(resume_directory / "last.pt")
    #     model.train(resume = True)

    # elif resume == False:
    #     print("Pre-training model on SOURCE set...")
    #     #training stage 1
    #     model = YOLO(CVmodel)
    #     checkpoint = 1
    #     result = os.path.join(source_result_dir, model_name)
    #     model.train( data = src_yaml,
    #                             epochs = src_eps, 
    #                             imgsz = imgsiz,
    #                             batch = 16,
    #                             project = result,
    #                             name = f"inspection_{model_name}", 
    #                             workers = 12, #add worker threads to improve training speed, useful for multi-GPU setup
    #                             cache = 'disk', #improve training speed by adding cache/using more memory
    #                             amp = True,
    #                             device = [-1, -1],

    #                             #Augmentation
    #                             lr0 = 0.01, lrf = 0.001,
    #                             optimizer     = "SGD",
    #                             momentum      = 0.937,
    #                             weight_decay  = 5e-4,
    #                             degrees       = 45,
    #                             shear         = 2.0,
    #                             mosaic        = 1.0,
    #                             mixup         = 0.20,
    #                             hsv_h         = 0.015,
    #                             hsv_s         = 0.70,
    #                             hsv_v         = 0.40,
    #                             patience      = 100  #for early stop if model stop improving
    #                         )

    # if (resume == False) or (resume == True and CVmodel == resume_model and checkpoint == 1) :
    #     #rectrieve the weight / pretrained model
    #     pr_model_pt = model.trainer.save_dir / "weights" / "best.pt"
    #     model.val(data = src_yaml, split = "test", device = [-1, -1], conf = 0.4, iou = 0.6,
    #                     imgsz = imgsiz, batch = 16,
    #                     save_json = True, plots = True, save_txt = True, save_conf = True,
    #                     project = src_eval_result_dir, name = f"inspection_{model_name}"
    #                    )
    #     if os.path.exists(pr_model_pt):
    #         print("Pretrained model exists")
    #     else:
    #         print("Pretrained model does not exists")


    # #-------------------------------------Continue training model on the TARGET dataset -------------------------------------

    # if resume == True and CVmodel == resume_model and checkpoint == 2:
    #     print("Resume training model on TARGET set...")
    #     model = YOLO(resume_directory / "last.pt")
    #     model.train(resume = True)
        
    # else:
    #     print("Continue training on TARGET set...")
    
    #     #training from scratch stage
    #     if CVmodel.split(".")[1] == "yaml":
    #         model_name = model_name + f"_scratch{tar_eps}"
    #         checkpoint = 2

            
    #         model = YOLO(CVmodel)
    #         result = os.path.join(target_result_dir, model_name)
    #         model.train( data = tar_yaml,
    #                                 epochs = tar_eps, 
    #                                 imgsz = imgsiz,
    #                                 batch = 8,
    #                                 project = result,
    #                                 name = f"inspection_{model_name}", 
    #                                 # freeze = 10, #freeze layers for transfer learning
    #                                 workers = 10, #add worker threads to improve training speed, useful for multi-GPU setup
    #                                 cache = 'disk', #improve training speed by adding cache/using more memory
    #                                 amp = True,
    #                                 device = [-1, -1],
    
    #                                 #Augmentation
    #                                 lr0 = 0.01, lrf = 0.01,
    #                                 # momentum        = 0.90,
    #                                 # shear           = 15,
    #                                 # degrees         = 15,
    #                                 # translate       = 0.1,
    #                                 # mosaic          = 0.5,
    #                                 # hsv_h           = 0.015,
    #                                 # hsv_s           = 0.50,
    #                                 # hsv_v           = 0.30,
    #                                 # label_smoothing = 0.05,
    #                                 # # class_weights   = cls_weights,  # ← from previous message
    #                                 # patience        = 50  #for early stop if model stop improving
    #                             )

    #     #transfer learning stage
    #     else:

    #         #target dataset stage, freeze backbone layers
    #         model = YOLO(CVmodel)
    #         checkpoint = 2
    #         model_name1 = model_name + "_freeze10"
    #         model_name2 = model_name + "_finetune"
    #         result = os.path.join(target_result_dir, model_name1)
    #         model.train( data = tar_yaml,
    #                                 epochs = tar_eps, 
    #                                 imgsz = imgsiz,
    #                                 batch = 8,
    #                                 project = result,
    #                                 name = f"inspection_{model_name1}", 
    #                                 freeze = 10, #freeze layers for transfer learning
    #                                 workers = 10, #add worker threads to improve training speed, useful for multi-GPU setup
    #                                 cache = 'disk', #improve training speed by adding cache/using more memory
    #                                 amp = True,
    #                                 device = [-1, -1],
    
    #                                 #Augmentation
    #                                 lr0 = 0.003, lrf = 0.0005,
    #                                 # momentum        = 0.90,
    #                                 # shear           = 15,
    #                                 # degrees         = 15,
    #                                 # translate       = 0.1,
    #                                 # mosaic          = 0.5,
    #                                 # hsv_h           = 0.015,
    #                                 # hsv_s           = 0.50,
    #                                 # hsv_v           = 0.30,
    #                                 # label_smoothing = 0.05,
    #                                 # patience        = 50  #for early stop if model stop improving
    #                             )
    #         pt = model.trainer.save_dir / "weights" / "best.pt"

    #         #fine-tuning stage, unfreeze backbone layers
    #         model = YOLO(pt)
    #         result = os.path.join(target_result_dir, model_name2)
    #         model.train( data = tar_yaml,
    #                                 epochs = fine_tune_eps, 
    #                                 imgsz = imgsiz,
    #                                 batch = 8,
    #                                 project = result,
    #                                 name = f"inspection_{model_name2}", 
    #                                 # freeze = 10, #freeze layers for transfer learning
    #                                 workers = 10, #add worker threads to improve training speed, useful for multi-GPU setup
    #                                 cache = 'disk', #improve training speed by adding cache/using more memory
    #                                 amp = True,
    #                                 device = [-1, -1],
    
    #                                 #Augmentation
    #                                 lr0 = 0.003, lrf = 0.1535,
    #                                 # momentum        = 0.90,
    #                                 # shear           = 15,
    #                                 # degrees         = 15,
    #                                 # translate       = 0.1,
    #                                 # mosaic          = 0.5,
    #                                 # hsv_h           = 0.015,
    #                                 # hsv_s           = 0.50,
    #                                 # hsv_v           = 0.30,
    #                                 # label_smoothing = 0.05,
    #                                 # patience        = 50  #for early stop if model stop improving
    #                             )
            

    # finish_pt = model.trainer.save_dir / "weights" / "best.pt"

     #-------------------------------------Testing model on TEST set -------------------------------------
    print("Evaluate on TEST set...")

    #-------------------------------------ONLY WHEN TRYING TO GRAB EVALUATION AGAIN AFTER TRAINING-------------------------------------
    finish_pt = os.path.join(target_result_dir, f"{model_name}_finetune/inspection_{model_name}_finetune/weights/best.pt")
    # finish_pt = os.path.join(target_result_dir, f"{model_name}_scratch{tar_eps}/inspection_{model_name}_scratch{tar_eps}/weights/best.pt")
    #----------------------------------------------------------------------------------------------------------------------------------
    
    model = YOLO(finish_pt)
    metrics = model.val(data = tar_yaml, split = "test", device = [-1, -1], conf = 0.4, iou = 0.6,
                        imgsz = imgsiz, batch = 16,
                        save_json = True, plots = True, save_txt = True, save_conf = True,
                        project = tar_eval_result_dir, name = f"inspection_{model_name}"
                       )
    
    return CVmodel, metrics
    
                        

In [30]:
#----------------------------------------- Add Class Weights to YAML -----------------------------------------
def compute_class_weights(dataset_root=Path("dataset/target"),  splits=("train", "val"), n_classes=7):
    """
    Scan YOLO label files and compute class_weight list
    using the "median-frequencing balancing" method .

    Returns: List[float] of length n_classes (0 if class never appears).
    """
    # get counts
    total_counts = Counter()
    for s in splits:
        for txt in (dataset_root / s / "labels").glob("*.txt"):
            with open(txt) as f:
                for line in f:
                    cls_id = int(line.split()[0])
                    total_counts[cls_id] += 1

    # build an array with class ids
    counts = np.zeros(n_classes, dtype=float)
    for cls_id, c in total_counts.items():
        counts[cls_id] = c

    if counts.sum() == 0:
        raise RuntimeError("No objects found – check dataset_root and splits!")

    # compute weights
    med = np.median(counts[counts > 0])          # ignore empty classes
    weights = np.where(counts > 0, med / counts, 0.0)

    print(f"\nCounts per class  : {counts.astype(int).tolist()}")
    print(f"Median of counts  : {med}")
    print(f"class_weights list: {weights.round(3).tolist()}")

    return weights.round(6).tolist()

In [31]:
#Computing class weights
cls_weights = compute_class_weights(
    dataset_root = Path("dataset/target"),  # or "dataset/source"
    splits       = ("train", "val"),        # add "test" if desired
    n_classes    = 7
)
cls_weights_cfg = {"class_weights" : cls_weights}


Counts per class  : [624, 621, 273, 1207, 3767, 2149, 1500]
Median of counts  : 1207.0
class_weights list: [1.934, 1.944, 4.421, 1.0, 0.32, 0.562, 0.805]


In [32]:
# Right now add class weights to ONLY target data.yaml
# Load the existing YAML file
yaml_file_path = target_data_yaml
with open(yaml_file_path, "r") as file:
    data = yaml.safe_load(file)

# Add or update the class_weights in the YAML content
data['class_weights'] = cls_weights_cfg['class_weights']

# Write the updated YAML back to the file
with open(yaml_file_path, "w") as file:
    yaml.dump(data, file)


In [24]:
#-----------------------------------------Initiate Training -----------------------------------------
test_models = ["yolov5n.yaml"]
result_maps = {}
result_precision = {}
result_recall = {}
result_f1 = {}
result_preprocess_speed = {}
result_inference_speed = {}
result_postprocess_speed = {}

#CHANGE EPOCHS HERE
source_epochs = 250
target_epochs = 600

for model in test_models:
    resume_model = model
    name, metrics = training_pipeline(CVmodel = model, 
                                      src_yaml = source_data_yaml, 
                                      tar_yaml = target_data_yaml,
                                      src_eps = source_epochs,
                                      tar_eps = target_epochs)
    result_maps[name] = metrics.box.map50
    result_precision[name] = metrics.box.p
    result_recall[name] = metrics.box.r
    result_f1[name] = metrics.box.f1
    result_preprocess_speed[name] = metrics.speed["preprocess"]
    result_inference_speed[name] = metrics.speed["inference"]
    result_postprocess_speed[name] = metrics.speed["postprocess"]
    

Evaluate on TEST set...
Searching for 2 idle GPUs with free memory >= 20.0% and free utilization >= 0.0%...
Selected idle CUDA devices [1, 2]
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:1 (Quadro RTX 6000, 24024MiB)
                                                        CUDA:2 (Quadro RTX 6000, 24024MiB)
YOLOv5n summary (fused): 84 layers, 2,504,309 parameters, 0 gradients, 7.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 947.9±345.4 MB/s, size: 42.2 KB)


val: Scanning /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/test/labels.cache... 537 images, 0 backgrounds, 0 corrupt: 100%|██████████| 537/537 [00:00<?, ?i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:06<00:00,  5.44it/s]


                   all        537       1842      0.801      0.673      0.763      0.473
              Birdnest         96         99      0.733      0.778      0.793      0.404
      Broken_Insulator         78        114      0.842      0.746      0.839      0.563
      Defective_Damper         33         66      0.771      0.561      0.705      0.436
   Flashover_Insulator         75        198       0.81      0.581      0.703      0.394
         Normal_Damper        168        597      0.809      0.616      0.735      0.447
     Normal_Insulators        189        486       0.76      0.685      0.727      0.496
Self-Exploded_Insulator        219        282      0.882      0.745      0.842      0.573
Speed: 0.6ms preprocess, 3.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/test_results/Target/inspection_yolov5n3/predictions.json...
Results saved to /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/test_results/Target/i

In [23]:
#----------------------------------------- Evaluation Matrices -----------------------------------------

# edit this list to match your dataset class order ---
class_names = [
    "Birdnest",
    "Broken_Insulator",
    "Defective_Damper",
    "Flashover_Insulator",
    "Normal_Damper",
    "Normal_Insulators",
    "Self-Exploded_Insulator",
]

def _to_array(x):
    return np.array(x, dtype=float)

print("="*72)
print("MODEL SUMMARY (overall)")
print("="*72)
header = f"{'Model':12s} {'mAP50':>7s} {'P(mean)':>8s} {'R(mean)':>8s} {'F1(mean)':>8s} {'Pre(ms)':>8s} {'Infer(ms)':>9s} {'Post(ms)':>8s}"
print(header)
print("-"*72)

# test_models list has filenames; use split to get the dict key names you stored
for model in test_models:
    key = model          # you stored metrics under this name
    map50 = float(result_maps[key])
    p_mean = float(_to_array(result_precision[key]).mean())
    r_mean = float(_to_array(result_recall[key]).mean())
    f1_mean = float(_to_array(result_f1[key]).mean())
    pre = float(result_preprocess_speed[key])
    infer = float(result_inference_speed[key])
    post = float(result_postprocess_speed[key])
    print(f"{key:12s} {map50:7.3f} {p_mean:8.3f} {r_mean:8.3f} {f1_mean:8.3f} {pre:8.2f} {infer:9.2f} {post:8.2f}")

print("\nDetailed per-class metrics (Precision / Recall / F1):")
for model in test_models:
    key = model
    P = _to_array(result_precision[key])
    R = _to_array(result_recall[key])
    F1 = _to_array(result_f1[key])
    print(f"\n[{key}]")
    print(f"{'Class':28s} {'P':>6s} {'R':>6s} {'F1':>6s}")
    for i, cname in enumerate(class_names):
        p = P[i] if i < len(P) else float("nan")
        r = R[i] if i < len(R) else float("nan")
        f1 = F1[i] if i < len(F1) else float("nan")
        print(f"{cname:28s} {p:6.3f} {r:6.3f} {f1:6.3f}")
print()

MODEL SUMMARY (overall)
Model          mAP50  P(mean)  R(mean) F1(mean)  Pre(ms) Infer(ms) Post(ms)
------------------------------------------------------------------------
yolov8n.yaml   0.770    0.802    0.680    0.733     0.62      2.49     2.15

Detailed per-class metrics (Precision / Recall / F1):

[yolov8n.yaml]
Class                             P      R     F1
Birdnest                      0.743  0.788  0.765
Broken_Insulator              0.845  0.719  0.777
Defective_Damper              0.745  0.576  0.650
Flashover_Insulator           0.809  0.576  0.673
Normal_Damper                 0.807  0.638  0.713
Normal_Insulators             0.776  0.726  0.750
Self-Exploded_Insulator       0.888  0.734  0.804



In [20]:
# Create the 'matrix_print' directory if it doesn't exist
output_dir = "matrix_print"
os.makedirs(output_dir, exist_ok=True)

# Define the file path for the output
output_file = os.path.join(output_dir, f"evaluation_metrics{target_epochs}.txt")

# Open the file in write mode
with open(output_file, "w") as f:
    # Write the header
    f.write("="*72 + "\n")
    f.write("MODEL SUMMARY (overall)\n")
    f.write("="*72 + "\n")
    
    header = f"{'Model':12s} {'mAP50':>7s} {'P(mean)':>8s} {'R(mean)':>8s} {'F1(mean)':>8s} {'Pre(ms)':>8s} {'Infer(ms)':>9s} {'Post(ms)':>8s}\n"
    f.write(header)
    f.write("-"*72 + "\n")

    # Write the metrics for each model in test_models
    for model in test_models:
        key = model 
        map50 = float(result_maps[key])
        p_mean = float(_to_array(result_precision[key]).mean())
        r_mean = float(_to_array(result_recall[key]).mean())
        f1_mean = float(_to_array(result_f1[key]).mean())
        pre = float(result_preprocess_speed[key])
        infer = float(result_inference_speed[key])
        post = float(result_postprocess_speed[key])
        f.write(f"{key:12s} {map50:7.3f} {p_mean:8.3f} {r_mean:8.3f} {f1_mean:8.3f} {pre:8.2f} {infer:9.2f} {post:8.2f}\n")

    f.write("\nDetailed per-class metrics (Precision / Recall / F1):\n")

    # Write per-class metrics for each model
    for model in test_models:
        key = model
        P = _to_array(result_precision[key])
        R = _to_array(result_recall[key])
        F1 = _to_array(result_f1[key])
        f.write(f"\n[{key}]\n")
        f.write(f"{'Class':28s} {'P':>6s} {'R':>6s} {'F1':>6s}\n")
        for i, cname in enumerate(class_names):
            p = P[i] if i < len(P) else float("nan")
            r = R[i] if i < len(R) else float("nan")
            f1 = F1[i] if i < len(F1) else float("nan")
            f.write(f"{cname:28s} {p:6.3f} {r:6.3f} {f1:6.3f}\n")

print(f"Evaluation metrics have been written to {output_file}")


Evaluation metrics have been written to matrix_print/evaluation_metrics600.txt


In [13]:
#-----------------------------------------Helpers fot Model Prediction--------------------------------
def model_prediction(model_name, model_pt_dir, pred_IoU, pred_threshold, pred_yaml):
    output_dir = Path.cwd()
    output_dir = os.path.join(output_dir, f"Predict_Result/{model_name}_Predictions")
    os.makedirs(output_dir, exist_ok=True)

    batch_sz = 16
    source_path = "/mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/test/images"
    model = YOLO(model_pt_dir)
    model.predict(source = source_path, conf = pred_threshold, imgsz = 640, iou = pred_IoU, 
                   batch=batch_sz, project = output_dir, name = f"{model_name}_run", 
                  save = True, save_txt = True, save_conf = True)

In [ ]:
# Run Model Prediction
model_choice = "yolo11n"
model_path = "/mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolo11n_finetune/inspection_yolo11n_finetune/weights/best.pt"
model_prediction(model_name = model_choice, model_pt_dir = model_path, pred_IoU = 0.5, pred_threshold = 0.5, pred_yaml = target_data_yaml)

In [29]:
#-----------------------------------------Helpers fot resume training purposes---------------------------------
def version_num(path: Path) -> int:
    suffix = path.name[len(prefix):]            # cut off the prefix
    return int(suffix) if suffix.isdigit() else 0


def find_recent_ver(target_result_dir):
    root         = Path(target_result_dir)          # or source_result_dir
    model_stem   = resume_model.split(".")[0]       # "yolov5n"
    prefix       = f"inspection_{model_stem}"       # "inspection_yolov5n"
    
    run_dirs = [p for p in (root / model_stem).iterdir() if p.is_dir() and p.name.startswith(prefix)]
    
    def version_num(path: Path) -> int:
        suffix = path.name[len(prefix):]            # cut off the prefix
        return int(suffix) if suffix.isdigit() else 0
    
    latest_run = max(run_dirs, key=version_num, default=None)
    if latest_run is None:
        raise FileNotFoundError(f"No runs found under {(root/model_stem).as_posix()}")
    
    resume_dir = latest_run / "weights"             # …/weights
    print("🔄  Will resume from:", resume_dir)
    return resume_dir

In [12]:
#-----------------------------------------For resume training purposes---------------------------------
source_result_dir = Path(source_result_dir)   # ← convert if they were str
target_result_dir = Path(target_result_dir)

checkpoint = 2


#retrieve the index of the resume model
resume_index = test_models.index(resume_model)


#retrieve the most recent version number
#check if is resuming to Pretrain stage or Target/finetune stage and set directory
most_recent_ver = 0
if checkpoint == 1:
    resume_dir = find_recent_ver(source_result_dir)
else:
    resume_dir = find_recent_ver(target_result_dir)

result_maps = {}
result_precision = {}
result_recall = {}
result_f1 = {}
result_preprocess_speed = {}
result_inference_speed = {}
result_postprocess_speed = {}

#resume training from most recent trained model
for model in test_models[resume_index:]:
    name, metrics = training_pipeline(CVmodel = model, 
                                      src_yaml = source_data_yaml, 
                                      tar_yaml = target_data_yaml,
                                      resume =True,
                                      resume_directory = resume_dir,
                                      src_eps = 250,
                                      tar_eps = 300)
    result_maps[name] = metrics.box.map50
    result_precision[name] = metrics.box.p
    result_recall[name] = metrics.box.r
    result_f1[name] = metrics.box.f1
    result_preprocess_speed[name] = metrics.speed["preprocess"]
    result_inference_speed[name] = metrics.speed["inference"]
    result_postprocess_speed[name] = metrics.speed["postprocess"]
        
        

Resume training model on TARGET set...
New https://pypi.org/project/ultralytics/8.3.170 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:1 (Quadro RTX 6000, 24024MiB)
                                                        CUDA:2 (Quadro RTX 6000, 24024MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/data.yaml, degrees=0.0, deterministic=True, device=1,2, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, 

train: Scanning /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/train/labels.cache... 2442 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2442/2442 [00:00<?, ?it/s]
train: Caching images (2.8GB Disk): 100%|██████████| 2442/2442 [00:00<00:00, 36911.21it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 884.5±541.4 MB/s, size: 50.7 KB)


val: Scanning /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/val/labels.cache... 510 images, 0 backgrounds, 0 corrupt: 100%|██████████| 510/510 [00:00<?, ?it/s]
val: Caching images (0.6GB Disk): 100%|██████████| 510/510 [00:00<00:00, 26453.00it/s]


Plotting labels to /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolov8n_scratch300/inspection_yolov8n_scratch300/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolov8n_scratch300/inspection_yolov8n_scratch300/weights/last.pt from epoch 94 to 300 total epochs
Image sizes 640 train, 640 val
Using 20 dataloader workers
Logging results to /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolov8n_scratch300/inspection_yolov8n_scratch300
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        510       1658      0.734      0.575      0.625      0.339

207 epochs completed in 1.687 hours.
Optimizer stripped from /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolov8n_scratch300/inspection_yolov8n_scratch300/weights/last.pt, 6.3MB
Optimizer stripped from /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolov8n_scratch300/inspection_yolov8n_scratch300/weights/best.pt, 6.3MB

Validating /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolov8n_scratch300/inspection_yolov8n_scratch300/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:1 (Quadro RTX 6000, 24024MiB)
                                                        CUDA:2 (Quadro RTX 6000, 24024MiB)
YOLOv8n summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:03<00:00, 19.94it/s]


                   all        510       1658      0.779      0.576       0.64      0.349
              Birdnest         96        102      0.854      0.873      0.859      0.469
      Broken_Insulator         78        126      0.756      0.437      0.564      0.326
      Defective_Damper         24         60      0.751      0.302      0.384       0.13
   Flashover_Insulator         69        180      0.691       0.55      0.589      0.299
         Normal_Damper        156        590      0.807      0.537      0.605       0.32
     Normal_Insulators        177        351      0.754      0.621      0.685       0.38
Self-Exploded_Insulator        216        249      0.842      0.715      0.793      0.521
Speed: 0.4ms preprocess, 1.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/target_results/yolov8n_scratch300/inspection_yolov8n_scratch300
Evaluate on TEST set...
Searching for 2 idle GPUs with free memory >= 20.0

val: Scanning /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/test/labels.cache... 537 images, 0 backgrounds, 0 corrupt: 100%|██████████| 537/537 [00:00<?, ?i
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:07<00:00,  4.80it/s]


                   all        537       1842      0.814      0.588      0.711      0.416
              Birdnest         96         99      0.783      0.727      0.733       0.39
      Broken_Insulator         78        114      0.785      0.447      0.633      0.403
      Defective_Damper         33         66      0.795       0.47      0.654      0.329
   Flashover_Insulator         75        198      0.831       0.52       0.68      0.355
         Normal_Damper        168        597      0.816      0.558      0.695      0.398
     Normal_Insulators        189        486        0.8      0.698      0.756      0.491
Self-Exploded_Insulator        219        282      0.887      0.699      0.823      0.544
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Saving /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/test_results/Target/inspection_yolov8n2/predictions.json...
Results saved to /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/test_results/Target/i